In [ ]:
# Tempo Sample Example Notebook
# Welcome! This notebook is intended to demonstrate the use of Tempo's Python API.
# It contains examples of:
#  - Streaming camera data
#  - Manipulating the scene at runtime
#  - Adding simulated cameras

In [ ]:
# Tempo imports
import tempo
import tempo.tempo_core as tc
import tempo.tempo_time as tt
import TempoTime.Time_pb2 as Time
import tempo.tempo_world as tw
import tempo.tempo_sensors as ts
import TempoImageUtils as tiu
import TempoScripting.Geometry_pb2 as Geometry

# Other imports
import asyncio
import cv2
import io
import math
import numpy as np
import random
import time

In [ ]:
# To connect to Tempo on a different machine
await tempo.set_server(address="0.0.0.0")

In [ ]:
# Find the Ego actor name
available_sensors_response = await ts.get_available_sensors()
ego_actor = [available_sensor.owner for available_sensor in available_sensors_response.available_sensors if "Unitree" in available_sensor.owner][0]

In [ ]:
# Stream color images from the front center camera
ego_color_image_stream = asyncio.create_task(tiu.stream_color_images("FrontCenterCamera", ego_actor))

In [ ]:
# End front center camera color image stream
_ = ego_color_image_stream.cancel()

In [ ]:
# Stream label images from the front center camera
ego_label_image_stream = asyncio.create_task(tiu.stream_label_images("FrontCenterCamera", ego_actor))

In [ ]:
# End front center camera label image stream
_ = ego_label_image_stream.cancel()

In [ ]:
# Stream depth images from the front center camera
ego_depth_image_stream = asyncio.create_task(tiu.stream_depth_images("FrontCenterCamera", ego_actor))

In [ ]:
# End front center camera depth image stream
_ = ego_depth_image_stream.cancel()

In [ ]:
# Decrease the brightness of the overhead directional light
all_actors_response = await tw.get_all_actors()
all_directional_lights = [actor.name for actor in all_actors_response.actors if "DirectionalLight" in actor.type]
for light in all_directional_lights:
    await tw.set_float_property(actor=light, component="LightComponent0", property="Intensity", value=.01)

In [ ]:
# Increase the brightness of the overhead directional light
all_actors_response = await tw.get_all_actors()
all_directional_lights = [actor.name for actor in all_actors_response.actors if "DirectionalLight" in actor.type]
for light in all_directional_lights:
    await tw.set_float_property(actor=light, component="LightComponent0", property="Intensity", value=1.0)

In [ ]:
# Restart color image stream from the front center camera
front_ego_color_image_stream = asyncio.create_task(tiu.stream_color_images("FrontCenterCamera", ego_actor))

In [ ]:
# Move the camera up 50cm
t = Geometry.Transform()
t.location.z = 0.5 # meters
await tw.set_component_transform(actor=ego_actor, component="FrontCenterCamera", transform=t)

In [ ]:
# Set camera resolution to 1080p (starts at 540p)
await tw.set_int_property(actor=ego_actor, component="FrontCenterCamera", property="SizeXY.X", value=1920)
await tw.set_int_property(actor=ego_actor, component="FrontCenterCamera", property="SizeXY.Y", value=1080)

In [ ]:
# End stream from front center camera
_ = front_ego_color_image_stream.cancel()

In [ ]:
# Add two new cameras, pointed 20 degrees left and right from FrontCenterCamera
t_left = Geometry.Transform()
t_left.rotation.y = math.pi / 9.0 # radians
_ = await tw.add_component(actor=ego_actor, parent="SkeletalMesh", type="TempoCamera", socket="Camera", transform=t_left, name="FrontLeftCamera")

t_right = Geometry.Transform()
t_right.rotation.y = -math.pi / 9.0 # radians
_ = await tw.add_component(actor=ego_actor, parent="SkeletalMesh", type="TempoCamera", socket="Camera", transform=t_right, name="FrontRightCamera")

In [ ]:
# Stream color images from the left and right cameras
left_ego_color_image_stream = asyncio.create_task(tiu.stream_color_images("FrontLeftCamera", ego_actor))
right_ego_color_image_stream = asyncio.create_task(tiu.stream_color_images("FrontRightCamera", ego_actor))

In [ ]:
# End left and right camera color image streams
_ = left_ego_color_image_stream.cancel()
_ = right_ego_color_image_stream.cancel()

In [ ]:
# Switch to headless mode (to save performance)
await tc.set_main_viewport_render_enabled(False)

In [ ]:
# !~Note~!
# What's with all the 'await's? Tempo comes with both sync and async versions of all its RPCs,
# and deduces the correct one to call from the calling context. A Jupyter notebook always
# runs in an async context, so you must await all calls to the Tempo API when using one.
# Otherwise, Tempo API calls will hang.
# In a non-Jupyter Python interpreter you would not need to do this.